# Merging `Osteil` and `Cuomo` data

In [ ]:
Sys.Date()

In [ ]:
knitr::opts_chunk$set(echo = TRUE, collapse = FALSE)

## Project description:

Osteil, and Cuomo data were analysed seperately. The name of the files to be used:

    20210127_Cuomo_SC3_filtered.ipynb
    20210324_Osteil4_SC3_filtered.ipynb
   

In [ ]:
suppressPackageStartupMessages({
    library(scMerge)
    library(SingleCellExperiment)
    library(scater)
    library(scran)
    library(dplyr)
    library(ggpubr)
    library(forcats)
    library(tidyr)
    library(data.table)
    library(Seurat)
    library(princurve)
    library(slingshot)
    library(UpSetR)
    library(ComplexHeatmap)
    library(biomaRt)
    library(BiocParallel)
    library(edgeR)
    library(SC3)
    library(scDblFinder)
    library(here)
    library(dittoSeq)
})

In [ ]:
baseDir <- here("20201211_scRNASeq_hiPSc/")
inputDir <- here("20201211_scRNASeq_hiPSc/input/")
outDir <- here("20201211_scRNASeq_hiPSc/output/")
cat(paste0("baseDir = ", baseDir))
cores = BiocParallel::MulticoreParam(workers = 32, progressbar = TRUE)
set1 <- "20210208_Cuomo"
set2 <- "20210324_Pierre"
set <- "20230223_Cuomo_Pierre"

## ggplot themes

the parameters for ggplot is set here.

In [ ]:
ggtheme_hx <- list(theme(axis.text.x=element_text(angle = 0, vjust = 0.5, 
                                                  size = 12, face="bold"),
          axis.text.y=element_text(angle = 0, vjust = 0.5, 
                                   size = 12, face="bold"),
          axis.title=element_text(size=14,face="bold"),
          legend.title = element_text(colour="black", 
                                      size=12, face="bold"),
          legend.text = element_text(colour="black", 
                                     size=12, face="bold"),
          strip.text = element_text(size = 20)))
ggtheme_vx <- list(theme(axis.text.x=element_text(angle = 90, vjust = 0.5, 
                                                  size = 12, face="bold"),
          axis.text.y=element_text(angle = 0, vjust = 0.5, 
                                   size = 12, face="bold"),
          axis.title=element_text(size=14,face="bold"),
          legend.title = element_text(colour="black", 
                                      size=12, face="bold"),
          legend.text = element_text(colour="black", 
                                     size=12, face="bold"),
          strip.text = element_text(size = 20)))

## Loading the data

In [ ]:
# loading Cuomo data 
sce1 <- readRDS(paste0(outDir, set1, "_sce.combined_hiSC_dbl_RD_SC3_slingshot.RDS"))
sce1

In [ ]:
# Loading Osteil data
sce2 <- readRDS(paste0(outDir, set2, "_sce.combined_hiSC_dbl_RD.RDS"))
sce2

In [ ]:
genes.sce1 <- assay(sce1, "counts") %>% 
    rownames()
genes.sce2 <- assay(sce2, "counts") %>% 
    rownames()

In [ ]:
geneIdx <- intersect(genes.sce1, genes.sce2)

In [ ]:
length(genes.sce1)
length(genes.sce2)
length(geneIdx)

In [ ]:
sce <- list(sce1, sce2)
sce <- lapply(sce, function(x) {
    x[geneIdx,]
})
sce

In [ ]:
assay(sce[[1]], "counts") %>% head()
assay(sce[[1]], "logcounts") %>% head()

In [ ]:
assay(sce[[2]], "counts") %>% head()
assay(sce[[2]], "logcounts") %>% head()

In [ ]:
days <- lapply(sce, function(x) {
    colData(x)$day %>% 
    levels()
})

In [ ]:
days
days <- union(days[[1]], days[[2]])
days

In [ ]:
colData(sce[[1]]) %>% head()

In [ ]:
sce.combined = scMerge::sce_cbind(sce_list = sce,
                                 method = "union",
                                 colData_names = c("orig.ident", "day", "donor"),
                                 batch_names = c("Cuomo", "Osteil"))

In [ ]:
sce.combined

In [ ]:
colData(sce.combined)$exp <- paste0(colData(sce.combined)$batch, "_", colData(sce.combined)$day)
colData(sce.combined)$batch <- NULL
colData(sce.combined)$exp <- as.factor(colData(sce.combined)$exp)

In [ ]:
sce.combined

In [ ]:
exp <- colData(sce.combined)$exp %>% 
    levels()
exp

In [ ]:
sce.list <- lapply(exp, function(x) {
  y <- subset(sce.combined, ,exp == x)
    return(y)
})
names(sce.list) <- exp

In [ ]:
sce.list

In [ ]:
# convert to matrix
sce.list <- lapply(sce.list, function(x) {
    assay(x) <- as.matrix(assay(x))
    logcounts(x) <- as.matrix(logcounts(x))
    return(x)
})

## Perform scMerge

In [ ]:
# get the list of the hgv for each sce seperately
hvg.list <- lapply(sce.list, function(x) {
  dec <- modelGeneVar(x)
  hvg <- getTopHVGs(dec, prop = 0.2)
})
sapply(hvg.list, length)

In [ ]:
hvg.day0 <- Reduce(intersect, hvg.list[c(1,5)])
hvg.day1 <- Reduce(intersect, hvg.list[c(2,6)])
hvg.day234 <- Reduce(intersect, hvg.list[c(3,4,7)])

In [ ]:
length(hvg.day0)
length(hvg.day1)
length(hvg.day234)

In [ ]:
colData(sce.list[[1]])

In [ ]:
sce.combine.day0 = scMerge::sce_cbind(sce_list = sce.list[c(1,5)],
                                 method = "union",
                                 colData_names = colnames(colData(sce.list[[1]])),
                                 batch_names = names(sce.list[c(1,5)]))

sce.combine.day1 = scMerge::sce_cbind(sce_list = sce.list[c(2,6)],
                                 method = "union",
                                 colData_names = colnames(colData(sce.list[[1]])),
                                 batch_names = names(sce.list[c(2,6)]))

sce.combine.day234 = scMerge::sce_cbind(sce_list = sce.list[c(3,4,7)],
                                 method = "union",
                                 colData_names = colnames(colData(sce.list[[1]])),
                                 batch_names = names(sce.list[c(3,4,7)]))

In [ ]:
colData(sce.combine.day0)$exp <- NULL
colData(sce.combine.day1)$exp <- NULL
colData(sce.combine.day234)$exp <- NULL

In [ ]:
colnames(sce.combine.day0) <- paste0(rownames(colData(sce.combine.day0)), "_", 1:nrow(colData(sce.combine.day0)))
colnames(sce.combine.day1) <- paste0(rownames(colData(sce.combine.day1)), "_", 1:nrow(colData(sce.combine.day1)))
colnames(sce.combine.day234) <- paste0(rownames(colData(sce.combine.day234)), "_", 1:nrow(colData(sce.combine.day234)))

In [ ]:
sce.combine.day0$batch <- as.factor(sce.combine.day0$batch)
sce.combine.day0$batch %>% levels()
sce.combine.day1$batch <- as.factor(sce.combine.day1$batch)
sce.combine.day1$batch %>% levels()
sce.combine.day234$batch <- as.factor(sce.combine.day234$batch)
sce.combine.day234$batch %>% levels()

## Unsupervised merging

In [ ]:
data("segList_ensemblGeneID", package = "scMerge")

ensembl <- useMart("ensembl", dataset="hsapiens_gene_ensembl")
bioMart_annot <- getBM(attributes=c('external_gene_name', 'ensembl_gene_id'),
             values = segList_ensemblGeneID$human$human_scSEG,
             mart = ensembl)

In [ ]:
segList_ensemblGeneID$human$human_scSEG %>% head()

In [ ]:
cat(paste0("Analysis started at: ", Sys.time(), "\n"))

rownames.ensembl <- bioMart_annot$ensembl_gene_id[bioMart_annot$external_gene_name %in% rownames(sce.combine.day0)]
names(rownames.ensembl) <- bioMart_annot$external_gene_name[bioMart_annot$external_gene_name %in% rownames(sce.combine.day0)]

scMerge.day0 = scMerge(
  sce_combine = sce.combine.day0,
  ctl = which(rownames.ensembl[rownames(sce.combine.day0)] %in% segList_ensemblGeneID$human$human_scSEG),
  kmeansK = c(3, 3),
  replicate_prop = 1,
  marker = hvg.day0,
  assay_name = "scMerge_unsupervised",
  BPPARAM = cores,
  verbose = TRUE)
cat(paste0("\nAnalysis finished at: ", Sys.time()))

In [ ]:
cat(paste0("Analysis started at: ", Sys.time(), "\n"))

rownames.ensembl <- bioMart_annot$ensembl_gene_id[bioMart_annot$external_gene_name %in% rownames(sce.combine.day1)]
names(rownames.ensembl) <- bioMart_annot$external_gene_name[bioMart_annot$external_gene_name %in% rownames(sce.combine.day1)]

scMerge.day1 = scMerge(
  sce_combine = sce.combine.day1,
  ctl = which(rownames.ensembl[rownames(sce.combine.day1)] %in% segList_ensemblGeneID$human$human_scSEG),
  kmeansK = c(3, 3),
  replicate_prop = 1,
  marker = hvg.day1,
  assay_name = "scMerge_unsupervised",
  BPPARAM = cores,
  verbose = TRUE)
cat(paste0("\nAnalysis finished at: ", Sys.time()))

In [ ]:
cat(paste0("Analysis started at: ", Sys.time(), "\n"))

rownames.ensembl <- bioMart_annot$ensembl_gene_id[bioMart_annot$external_gene_name %in% rownames(sce.combine.day234)]
names(rownames.ensembl) <- bioMart_annot$external_gene_name[bioMart_annot$external_gene_name %in% rownames(sce.combine.day234)]

scMerge.day234 = scMerge(
  sce_combine = sce.combine.day234,
  ctl = which(rownames.ensembl[rownames(sce.combine.day234)] %in% segList_ensemblGeneID$human$human_scSEG),
  kmeansK = c(5,5,5),
  replicate_prop = 1,
  marker = hvg.day234,
  assay_name = "scMerge_unsupervised",
  BPPARAM = cores,
  verbose = TRUE)

cat(paste0("\nAnalysis finished at: ", Sys.time()))

In [ ]:
scMerge.day234

In [ ]:
sce.combined = scMerge::sce_cbind(sce_list = sce.list,
                                 method = "union",
                                 colData_names = colnames(colData(sce.list[[1]])),
                                 batch_names = names(sce.list))

In [ ]:
colnames(sce.combined) <- paste0(rownames(colData(sce.combined)), "_", 1:nrow(colData(sce.combined)))

In [ ]:
colnames(sce.combined) %>% head()

In [ ]:
sce.combined$batch <- as.factor(sce.combined$batch)
sce.combined$batch %>% levels()

### Downstream analysis - scMerge all days as day0, day1, day234

In [ ]:
sce.scmerge.list1 <- list(scMerge.day0, scMerge.day1, scMerge.day234)

In [ ]:
sce.scmerge.list1 <- lapply(sce.scmerge.list1, function(x){
    x$exp <- x$batch
    return(x)
})

In [ ]:
sce.scmerge.list1[[2]]

In [ ]:
sce.scmerge.days01234 = scMerge::sce_cbind(sce_list = sce.scmerge.list1,
                                 method = "union",
                                 exprs = c("counts", "logcounts", "scMerge_unsupervised"),
                                 colData_names = colnames(colData(sce.scmerge.list1[[1]])),
                                 batch_names = NULL)

In [ ]:
sce.scmerge.days01234$batch <- sce.scmerge.days01234$exp
sce.scmerge.days01234$exp <- NULL

In [ ]:
colData(sce.scmerge.days01234)

In [ ]:
sce.scmerge.days01234

In [ ]:
set.seed(2020)
pca <- calculatePCA(sce.scmerge.days01234, exprs_values="scMerge_unsupervised", ntop = 500)
tsne <- calculateTSNE(sce.scmerge.days01234, exprs_values="scMerge_unsupervised", ntop = 500)
umap <- calculateUMAP(sce.scmerge.days01234, exprs_values="scMerge_unsupervised", ntop = 500)

reducedDims(sce.scmerge.days01234) <- SimpleList(PCA = pca, 
                                    TSNE = tsne,
                                    UMAP = umap)

In [ ]:
set_ind <- "20210306_scMerge_ind"
saveRDS(sce.scmerge.days01234, file = paste0(outDir, set_ind, "_hiSC_RD.RDS"))
cat("File saved at: \n", paste0(outDir, set_ind, "_hiSC_RD.RDS"))

In [ ]:
set_ind <- "20210306_scMerge_ind"
sce.scmerge.days01234 <- readRDS(paste0(outDir, set_ind, "_hiSC_RD.RDS"))

In [ ]:
sce.scmerge.days01234

In [ ]:
types <- c("day", "batch")
types

In [ ]:
set

In [ ]:
paste0(outDir, set, "_Part6_PCA_types_merged_op5.pdf")

In [ ]:
options(repr.plot.width=10, repr.plot.height=7)
plots <- lapply(types, function(x){
    dittoDimPlot(sce.scmerge.days01234, x, 
             reduction.use = "PCA", 
             legend.title = x, main = x, order = "decreasing") + ggtheme_hx
    
})
plots
pdf(file = paste0(outDir, set, "_Part6_PCA_types_merged_op5.pdf"), width = 10, height = 7)
plots
dev.off()

In [ ]:
options(repr.plot.width=10, repr.plot.height=7)
plots <- lapply(types, function(x){
    dittoDimPlot(sce.scmerge.days01234, x, 
             reduction.use = "PCA", 
             legend.title = x, main = x, order = "decreasing", opacity = 0.5) + ggtheme_hx
    
})
plots
pdf(file = paste0(outDir, set, "_Part6_PCA_types_merged_or_op5.pdf"), width = 10, height = 7)
plots
dev.off()

In [ ]:
options(repr.plot.width=10, repr.plot.height=7)
plots <- lapply(types, function(x){
    dittoDimPlot(sce.scmerge.days01234, x, 
             reduction.use = "UMAP", 
             legend.title = x, main = x, order = "decreasing", opacity = 0.5) + ggtheme_hx
    
})
plots
pdf(file = paste0(outDir, set, "_Part6_UMAP_types_merged_op5.pdf"), width = 10, height = 7)
plots
dev.off()

In [ ]:
options(repr.plot.width=10, repr.plot.height=7)
plots <- lapply(types, function(x){
    dittoDimPlot(sce.scmerge.days01234, x, 
             reduction.use = "TSNE", 
             legend.title = x, main = x, order = "decreasing", opacity = 0.5) + ggtheme_hx
    
})
plots
pdf(file = paste0(outDir, set, "_Part6_TSNE_types_merged_op5.pdf"), width = 10, height = 7)
plots
dev.off()

In [ ]:
plotPCA(sce.scmerge.days01234, colour_by = "batch")

In [ ]:
plotTSNE(sce.scmerge.days01234, colour_by = "batch")

In [ ]:
plotUMAP(sce.scmerge.days01234, colour_by = "batch")

In [ ]:
options(repr.plot.width=19, repr.plot.height=9)
plots <- lapply(types, function(x){
    dittoDimPlot(sce.scmerge.days01234, x, 
             reduction.use = "PCA", split.by = "day", 
             legend.title = x, main = x) + ggtheme_hx
    
})
plots

In [ ]:
options(repr.plot.width=19, repr.plot.height=9)
plots <- lapply(types, function(x){
    dittoDimPlot(sce.scmerge.days01234, x, 
             reduction.use = "UMAP", split.by = "day", 
             legend.title = x, main = x) + ggtheme_hx
    
})
plots

In [ ]:
options(repr.plot.width=19, repr.plot.height=9)
plots <- lapply(types, function(x){
    dittoDimPlot(sce.scmerge.days01234, x, 
             reduction.use = "TSNE", split.by = "day", 
             legend.title = x, main = x) + ggtheme_hx
    
})
plots

## Figures merged

In [ ]:
sce.scmerge.days01234

In [ ]:
options(repr.plot.width=9, repr.plot.height=7)
plotPCA(sce.scmerge.days01234, colour_by = "batch")

In [ ]:
plotTSNE(sce.scmerge.days01234, colour_by = "batch")

In [ ]:
plotUMAP(sce.scmerge.days01234, colour_by = "batch")

## Figures merged

In [ ]:
sce.scmerge.days01234

In [ ]:
PCA1 <- as.matrix(reducedDims(sce.scmerge.days01234)[["PCA"]][,1] * -1)
colnames(PCA1) <- "pc1"
PCA1 %>% head()

In [ ]:
UMAP1 <- as.matrix(reducedDims(sce.scmerge.days01234)[["UMAP"]][,1] * -1)
colnames(UMAP1) <- "umap1"
UMAP1 %>% head()

In [ ]:
TSNE1 <- as.matrix(reducedDims(sce.scmerge.days01234)[["TSNE"]][,2] * -1)
colnames(TSNE1) <- "tsne1"
TSNE1 %>% head()

In [ ]:
reducedDims(sce.scmerge.days01234)[["PCA1"]] <- PCA1
reducedDims(sce.scmerge.days01234)[["UMAP1"]] <- UMAP1
reducedDims(sce.scmerge.days01234)[["TSNE1"]] <- TSNE1

In [ ]:
#Run slingshot
slingshot_pca <- slingshot(sce.scmerge.days01234, reducedDim="PCA1")
slingshot_umap <- slingshot(sce.scmerge.days01234, reducedDim="UMAP1")
slingshot_tsne <- slingshot(sce.scmerge.days01234, reducedDim="TSNE1")

In [ ]:
colData(slingshot_pca)

In [ ]:
# Let's scale the slingPseudotime_1 data
library(scales)
colData(slingshot_pca)$scaled_slingPseudotime_1 <- rescale(colData(slingshot_pca)$slingPseudotime_1, c(0,1))
colData(slingshot_tsne)$scaled_slingPseudotime_1 <- rescale(colData(slingshot_tsne)$slingPseudotime_1, c(0,1))
colData(slingshot_umap)$scaled_slingPseudotime_1 <- rescale(colData(slingshot_umap)$slingPseudotime_1, c(0,1))
sce <- slingshot_pca

colData(sce)$slingPseudotime_1 <- NULL
colData(sce)$scaled_slingPseudotime_1 <- NULL

colData(sce)$slingPseudotime_pca <- colData(slingshot_pca)$slingPseudotime_1
colData(sce)$slingPseudotime_tsne <- colData(slingshot_tsne)$slingPseudotime_1
colData(sce)$slingPseudotime_umap <- colData(slingshot_umap)$slingPseudotime_1

colData(sce)$scaled_time_pca <- colData(slingshot_pca)$scaled_slingPseudotime_1
colData(sce)$scaled_time_tsne <- colData(slingshot_tsne)$scaled_slingPseudotime_1
colData(sce)$scaled_time_umap <- colData(slingshot_umap)$scaled_slingPseudotime_1

colData(sce)

In [ ]:
set_ind <- "20210406_scMerge_ind_ExFA2"
saveRDS(sce, file = paste0(outDir, set_ind, "_hiSC_dbl_RD_slingshot.RDS"))
cat("File saved at: \n", paste0(outDir, set_ind, "_hiSC_dbl_RD_slingshot.RDS"))

In [ ]:
set_ind <- "20210406_scMerge_ind_ExFA2"
sce <- readRDS(paste0(outDir, set_ind, "_hiSC_dbl_RD_slingshot.RDS"))

In [ ]:
meta <- as.data.frame(colData(sce))

In [ ]:
meta %>% head()

In [ ]:
days <- meta$day %>% 
        levels()
days

In [ ]:
days[1:3]

In [ ]:
meta.day0 <- meta %>%
        dplyr::filter(day == "day0")

meta.day1 <- meta %>%
        dplyr::filter(day == "day1")

meta.day2 <- meta %>%
        dplyr::filter(day == "day2" | day == "day4")

meta.day3 <- meta %>%
        dplyr::filter(day == "day3" | day == "day4")

meta.days <- list(meta.day0, meta.day1, meta.day2, meta.day3)
names(meta.days) <- days[1:4]

In [ ]:
lapply(meta.days, head)

In [ ]:
Pseudotime_pca <- lapply(meta.days, function(x){
    with(x, tapply(scaled_time_pca, donor, mean))
})
Pseudotime_umap <- lapply(meta.days, function(x){
    with(x, tapply(scaled_time_umap, donor, mean))
})
Pseudotime_tsne <- lapply(meta.days, function(x){
    with(x, tapply(scaled_time_tsne, donor, mean))
})

In [ ]:
lapply(Pseudotime_pca, dim)

In [ ]:
lapply(Pseudotime_umap, dim)

In [ ]:
lapply(Pseudotime_tsne, dim)

In [ ]:
# Let's remove the donors which they had no values.
Pseudotime_pca <- lapply(Pseudotime_pca, function(x){
    x[!is.na(x)]
})
Pseudotime_umap <- lapply(Pseudotime_umap, function(x){
    x[!is.na(x)]
})
Pseudotime_tsne <- lapply(Pseudotime_tsne, function(x){
    x[!is.na(x)]
})

In [ ]:
lapply(Pseudotime_pca, dim)

In [ ]:
lapply(Pseudotime_umap, dim)

In [ ]:
lapply(Pseudotime_tsne, dim)

In [ ]:
lapply(c(1:length(days[1:4])), function(x){
    plot(1:length(sort(Pseudotime_pca[[x]], decreasing=T)), 
     sort(Pseudotime_pca[[x]], decreasing=T), 
     main=paste0("Pseudotime gradient with donors " ,days[x]), 
         xlab = "Donors ordered by Pseudotime", 
         ylab = "Pseudotime based on PCA")
    points(match(Pseudotime_pca[[x]]["FA3"], 
             sort(Pseudotime_pca[[x]], decreasing=T)), Pseudotime_pca[[x]]["FA3"],
       pch=19, col="blue")
    points(match(Pseudotime_pca[[x]]["MB2"], 
             sort(Pseudotime_pca[[x]], decreasing=T)), Pseudotime_pca[[x]]["MB2"],
       pch=19, col="green")
    points(match(Pseudotime_pca[[x]]["MB3"], 
             sort(Pseudotime_pca[[x]], decreasing=T)), Pseudotime_pca[[x]]["MB3"],
       pch=19, col="orange")
})

In [ ]:
lapply(c(1:length(days[1:4])), function(x){
    plot(1:length(sort(Pseudotime_umap[[x]], decreasing=T)), 
     sort(Pseudotime_umap[[x]], decreasing=T), 
     main=paste0("Pseudotime gradient with donors " ,days[x]), 
         xlab = "Donors ordered by Pseudotime", 
         ylab = "Pseudotime based on UMAP")
    points(match(Pseudotime_umap[[x]]["FA3"], 
             sort(Pseudotime_umap[[x]], decreasing=T)), Pseudotime_umap[[x]]["FA3"],
       pch=19, col="blue")
    points(match(Pseudotime_umap[[x]]["MB2"], 
             sort(Pseudotime_umap[[x]], decreasing=T)), Pseudotime_umap[[x]]["MB2"],
       pch=19, col="green")
    points(match(Pseudotime_umap[[x]]["MB3"], 
             sort(Pseudotime_umap[[x]], decreasing=T)), Pseudotime_umap[[x]]["MB3"],
       pch=19, col="orange")
})

In [ ]:
lapply(c(1:length(days[1:4])), function(x){
    plot(1:length(sort(Pseudotime_tsne[[x]], decreasing=T)), 
     sort(Pseudotime_tsne[[x]], decreasing=T), 
     main=paste0("Pseudotime gradient with donors " ,days[x]), 
         xlab = "Donors ordered by Pseudotime", 
         ylab = "Pseudotime based on TSNE")
    points(match(Pseudotime_tsne[[x]]["FA3"], 
             sort(Pseudotime_tsne[[x]], decreasing=T)), Pseudotime_tsne[[x]]["FA3"],
       pch=19, col="blue")
    points(match(Pseudotime_tsne[[x]]["MB2"], 
             sort(Pseudotime_tsne[[x]], decreasing=T)), Pseudotime_tsne[[x]]["MB2"], 
       pch=19, col="green")
    points(match(Pseudotime_tsne[[x]]["MB3"], 
             sort(Pseudotime_tsne[[x]], decreasing=T)), Pseudotime_tsne[[x]]["MB3"],
       pch=19, col="orange")
})

In [ ]:
# Let's combine the tables
flat.Pseudotime_pca <- as.data.frame(bind_rows(Pseudotime_pca, .id = "days"))
flat.Pseudotime_umap <- as.data.frame(bind_rows(Pseudotime_umap, .id = "days"))
flat.Pseudotime_tsne <- as.data.frame(bind_rows(Pseudotime_tsne, .id = "days"))

In [ ]:
flat.Pseudotime_pca

In [ ]:
flat.Pseudotime_umap

In [ ]:
flat.Pseudotime_tsne

In [ ]:
meltdf <- as.data.frame(melt(flat.Pseudotime_pca,id="days"))
ggplot(meltdf,aes(x=days,y=value,colour=variable,group=variable)) + 
  geom_line(show.legend = FALSE) + 
  geom_point(show.legend = FALSE) +
  theme_bw() +
  labs(title="Time Series for", 
       subtitle="All donors for all timepoints", 
       caption="Osteil data merged with Cuomo et al 2020", 
       x = "Days", 
       y = "Pseudotime based on PCA")

In [ ]:
Pseudotime_pca <- lapply(meta.days, function(x){
    mean <- with(x, tapply(scaled_time_pca, donor, mean))
    SD <- with(x, tapply(scaled_time_pca, donor, sd))
    df <- cbind(mean, SD) %>% 
        as.data.frame() %>%
        arrange(desc(mean))
    return(df)
})

In [ ]:
days
# Pseudotime_pca
rownames(Pseudotime_pca$day0)

## Extracting top/buttom 20 donors

In [ ]:
donor.pca.top20 <- lapply(Pseudotime_pca, function(x){
    s <- x %>%
        arrange(desc(mean)) %>%
        slice_head(n = 20) %>%
        rownames()
    s <- union(s, c("FA3", "MB2", "MB3"))
    return(s)
})
names(donor.pca.top20) <- paste0(days[1:4], "-top20")
donor.pca.top20

In [ ]:
donor.pca.low20 <- lapply(Pseudotime_pca, function(x){
      s <-  x %>%
        arrange(mean) %>%
        slice_head(n = 20) %>%
        rownames()
    s <- union(s, c("FA3", "MB2", "MB3"))
    return(s)
})
names(donor.pca.low20) <- paste0(days[1:4], "-low20")
donor.pca.low20

In [ ]:
donor.pca.min.max.20 <- lapply(c(1:4), function(x){
    union(donor.pca.top20[[x]], donor.pca.low20[[x]])
})
names(donor.pca.min.max.20) <- paste0(days[1:4], "-min.max.20")
donor.pca.min.max.20

In [ ]:
meltdf <- as.data.frame(melt(flat.Pseudotime_pca,id="days"))
meltdf.top20 <- lapply(donor.pca.top20, function(x){
    meltdf[is.element(meltdf$variable, x), ]
})

meltdf.low20 <- lapply(donor.pca.low20, function(x){
    meltdf[is.element(meltdf$variable, x), ]
})
meltdf.min.max.20 <- lapply(donor.pca.min.max.20, function(x){
    meltdf[is.element(meltdf$variable, x), ]
})

In [ ]:
lapply(c(1:4), function(x){
    ggplot(meltdf.top20[[x]],aes(x=days,y=value,colour=variable,group=variable)) + 
      geom_line(show.legend = TRUE) + 
      geom_point(show.legend = TRUE) +
      theme_bw() +
      labs(title="Time Series", 
           subtitle=paste0("Top 20 donors for ", names(meltdf.top20)[x]), 
           caption="Osteil merged with Cuomo et al 2020", 
           x = "Days", 
           y = "Pseudotime based on PCA")
})

In [ ]:
lapply(c(1:4), function(x){
    ggplot(meltdf.low20[[x]],aes(x=days,y=value,colour=variable,group=variable)) + 
      geom_line(show.legend = TRUE) + 
      geom_point(show.legend = TRUE) +
      theme_bw() +
      labs(title="Time Series", 
           subtitle=paste0("Low 20 donors for ", names(meltdf.low20)[x]), 
           caption="Osteil merged with Cuomo et al 2020", 
           x = "Days", 
           y = "Pseudotime based on PCA")
})

In [ ]:
lapply(c(1:4), function(x){
    ggplot(meltdf.min.max.20[[x]],aes(x=days,y=value,colour=variable,group=variable)) + 
      geom_line(show.legend = TRUE) + 
      geom_point(show.legend = TRUE) +
      theme_bw() +
      labs(title="Time Series", 
           subtitle=paste0("Donors for ", names(meltdf.min.max.20)[x]), 
           caption="Osteil merged with Cuomo et al 2020", 
           y = "Pseudotime based on PCA")
})

In [ ]:
sce

In [ ]:
colData(sce)

In [ ]:
colData(sce)$topDay0 <- colData(sce)$donor
colData(sce)$topDay1 <- colData(sce)$donor
colData(sce)$topDay2 <- colData(sce)$donor
colData(sce)$topDay3 <- colData(sce)$donor

In [ ]:
colData(sce)

In [ ]:
donor.pca.top20.Cuomo <- lapply(Pseudotime_pca, function(x){
    s <- x %>%
        arrange(desc(mean)) %>%
        slice_head(n = 20) %>%
        rownames()
#     s <- union(s, c("FA2", "FA3", "MB2", "MB3"))
    return(s)
})
names(donor.pca.top20.Cuomo) <- paste0(days[1:4], "-top20")
donor.pca.top20.Cuomo

In [ ]:
donor.pca.low20.Cuomo <- lapply(Pseudotime_pca, function(x){
      s <-  x %>%
        arrange(mean) %>%
        slice_head(n = 20) %>%
        rownames()
#     s <- union(s, c("FA2", "FA3", "MB2", "MB3"))
    return(s)
})
names(donor.pca.low20.Cuomo) <- paste0(days[1:4], "-low20")
donor.pca.low20.Cuomo

In [ ]:
colData(sce)$topDay0 <- gsub(paste(donor.pca.top20.Cuomo$`day0-top20`, collapse="|"), "day0_top20", colData(sce)$topDay0)
colData(sce)$topDay1  <- gsub(paste(donor.pca.top20.Cuomo$`day1-top20`, collapse="|"), "day1_top20", colData(sce)$topDay1)
colData(sce)$topDay2  <- gsub(paste(donor.pca.top20.Cuomo$`day2-top20`, collapse="|"), "day2_top20", colData(sce)$topDay2)
colData(sce)$topDay3  <- gsub(paste(donor.pca.top20.Cuomo$`day3-top20`, collapse="|"), "day3_top20", colData(sce)$topDay3)

In [ ]:
paste(donor.pca.top20.Cuomo$`day3-top20`, collapse="|")

In [ ]:
colData(sce)$topDay0 <- gsub(paste(donor.pca.low20.Cuomo$`day0-low20`, collapse="|"), "day0_low20", colData(sce)$topDay0)
colData(sce)$topDay1  <- gsub(paste(donor.pca.low20.Cuomo$`day1-low20`, collapse="|"), "day1_low20", colData(sce)$topDay1)
colData(sce)$topDay2  <- gsub(paste(donor.pca.low20.Cuomo$`day2-low20`, collapse="|"), "day2_low20", colData(sce)$topDay2)
colData(sce)$topDay3  <- gsub(paste(donor.pca.low20.Cuomo$`day3-low20`, collapse="|"), "day3_low20", colData(sce)$topDay3)

In [ ]:
levels(as.factor(colData(sce)$topDay0))
levels(as.factor(colData(sce)$topDay1))
levels(as.factor(colData(sce)$topDay2))
levels(as.factor(colData(sce)$topDay3))

In [ ]:
table(sce$topDay0)

In [ ]:
table(sce$topDay1)

In [ ]:
table(sce$topDay2)

In [ ]:
table(sce$topDay3)

In [ ]:
sce.day0.top <- subset(sce, , topDay0=="day0_top20")
sce.day1.top <- subset(sce, , topDay1=="day1_top20")
sce.day2.top <- subset(sce, , topDay2=="day2_top20")
sce.day3.top <- subset(sce, , topDay3=="day3_top20")
sce.day0.low <- subset(sce, , topDay0=="day0_low20")
sce.day1.low <- subset(sce, , topDay1=="day1_low20")
sce.day2.low <- subset(sce, , topDay2=="day2_low20")
sce.day3.low <- subset(sce, , topDay3=="day3_low20")

In [ ]:
table(sce.day0.top$topDay0)
table(sce.day0.low$topDay0)
table(sce.day1.top$topDay1)
table(sce.day1.low$topDay1)
table(sce.day2.top$topDay2)
table(sce.day2.low$topDay2)
table(sce.day3.top$topDay3)
table(sce.day3.low$topDay3)

In [ ]:
sce.topDays <- list(sce.day0.top, sce.day1.top, sce.day2.top, sce.day3.top,sce.day0.low, sce.day1.low, sce.day2.low, sce.day3.low)
topNames <- c("sce.day0.top", "sce.day1.top", "sce.day2.top", "sce.day3.top", "sce.day0.low", "sce.day1.low", "sce.day2.low", "sce.day3.low")
names(sce.topDays) <- topNames

In [ ]:
suppressWarnings({suppressMessages({
    seuratObj.topDays <- lapply(names(sce.topDays), function(x){
        as.Seurat(sce.topDays[[x]], counts = "counts", data = "logcounts", project = x)
    })
})})
names(seuratObj.topDays) <- topNames

In [ ]:
seuratObj.topDays

In [ ]:
seuratObj.topDays.merged <- merge(seuratObj.topDays[[1]], y = c(seuratObj.topDays[[2]], seuratObj.topDays[[3]], seuratObj.topDays[[4]],
                                                               seuratObj.topDays[[5]],seuratObj.topDays[[6]],seuratObj.topDays[[7]],seuratObj.topDays[[8]]), 
                                  add.cell.ids = topNames, 
                                  project = "tops")

In [ ]:
levels(seuratObj.topDays.merged)

In [ ]:
grep("day0.*low", levels(seuratObj.topDays.merged), value=TRUE)

In [ ]:
group1 = grep("day0.*top", levels(seuratObj.topDays.merged), value=TRUE)
group2 = grep("day0.*low", levels(seuratObj.topDays.merged), value=TRUE)

cat("\nComparison:\t", group1, "\tversus\t", group2, "\n")

day0_markers <- FindMarkers(seuratObj.topDays.merged, 
                                  slot = "data", 
                                  ident.1 = group1, 
                                  ident.2 = group2, 
                                  test.use = "wilcox", 
                                  min.pct = 0.1)

cat("\nNumber of significant genes: ", nrow(day0_markers), "\n\n")

day0_markers %>% head(10)

In [ ]:
group1 = grep("day1.*top", levels(seuratObj.topDays.merged), value=TRUE)
group2 = grep("day1.*low", levels(seuratObj.topDays.merged), value=TRUE)

cat("\nComparison:\t", group1, "\tversus\t", group2, "\n")

day1_markers <- FindMarkers(seuratObj.topDays.merged, 
                                  slot = "data", 
                                  ident.1 = group1, 
                                  ident.2 = group2, 
                                  test.use = "wilcox", 
                                  min.pct = 0.1)

cat("\nNumber of significant genes: ", nrow(day1_markers), "\n\n")

day1_markers %>% head(10)

In [ ]:
group1 = grep("day2.*top", levels(seuratObj.topDays.merged), value=TRUE)
group2 = grep("day2.*low", levels(seuratObj.topDays.merged), value=TRUE)

cat("\nComparison:\t", group1, "\tversus\t", group2, "\n")

day2_markers <- FindMarkers(seuratObj.topDays.merged, 
                                  slot = "data", 
                                  ident.1 = group1, 
                                  ident.2 = group2, 
                                  test.use = "wilcox", 
                                  min.pct = 0.1)

cat("\nNumber of significant genes: ", nrow(day2_markers), "\n\n")

day2_markers %>% head(10)

In [ ]:
group1 = grep("day3.*top", levels(seuratObj.topDays.merged), value=TRUE)
group2 = grep("day3.*low", levels(seuratObj.topDays.merged), value=TRUE)

cat("\nComparison:\t", group1, "\tversus\t", group2, "\n")

day3_markers <- FindMarkers(seuratObj.topDays.merged, 
                                  slot = "data", 
                                  ident.1 = group1, 
                                  ident.2 = group2, 
                                  test.use = "wilcox", 
                                  min.pct = 0.1)

cat("\nNumber of significant genes: ", nrow(day3_markers), "\n\n")

day3_markers %>% head(10)

In [ ]:
results.list <- lapply(ls(pattern = ".*_markers"), get)
names(results.list) <- ls(pattern = ".*_markers")
lapply(results.list, head)

In [ ]:
resultsTblListAll <- lapply(results.list, function(x){
    x %>%
    tibble::rownames_to_column(var = "GeneID") %>%
    arrange(p_val_adj)
})

In [ ]:
resultsTblList <- lapply(resultsTblListAll, function(x){
    x %>%
    dplyr::filter(p_val_adj < 0.05 & abs(avg_log2FC) > 0.5)
})

In [ ]:
flatDE <- bind_rows(resultsTblList, .id = "Comparison")
writexl::write_xlsx(flatDE, paste0(outDir, set, "_DEGs_FDR5.xlsx"))

flatAll <- bind_rows(resultsTblListAll, .id = "Comparison")
writexl::write_xlsx(flatAll, paste0(outDir, set, "_DEGs_all.xlsx"))

In [ ]:
sessionInfo()